In [1]:
import torch
from torch import Tensor
import os
import sys
sys.path.append("..")
import dlc_practical_prologue as prologue
os.environ["PYTORCH_DATA_DIR"]="data"


In [2]:
train_input, train_target, test_input, test_target = prologue.load_data()

0it [00:00, ?it/s]* Using MNIST
9920512it [00:05, 1970789.59it/s]                             
Extracting data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to data/mnist/MNIST/raw
0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz
32768it [00:00, 83657.64it/s]            
0it [00:00, ?it/s]Extracting data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to data/mnist/MNIST/raw
1654784it [00:00, 1898918.09it/s]                            
0it [00:00, ?it/s]Extracting data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to data/mnist/MNIST/raw
8192it [00:00, 18010.24it/s]            
Extracting data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/mnist/MNIST/raw
Processing...
Done!
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [3]:
def nearest_classification(train_input, train_target, x):
    distances = (train_input - x[None,:]).pow(2).mean(axis=-1)
    return train_target[torch.argmin(distances)]


print(nearest_classification(train_input, train_target, test_input[0]))



tensor(7)


In [29]:
def compute_nb_errors(train_input, train_target, test_input, test_target, mean=None, proj=None):
    ntest = test_input.shape[0]
    prediction = torch.empty(ntest)
    accuracy = torch.empty(ntest)
    if mean is not None:
        train_input-=mean
        test_input-=mean
    if proj is not None:
        train_input=train_input.mm(proj.t())
        test_input=test_input.mm(proj.t())
    for i in range(ntest):
        prediction[i] = nearest_classification(train_input, train_target, test_input[i])
    accuracy = prediction!=test_target
    return accuracy.int().sum(), (accuracy).float().mean()



In [30]:
compute_nb_errors(train_input, train_target, test_input, test_target, mean=None, proj = None)

(tensor(746), tensor(0.7460))

In [18]:
def PCA(x):
    
    mean = x.mean(axis=0)
    std_x = x - mean
    cov = std_x.t().mm(std_x)
    eival, eivect = torch.eig(cov, eigenvectors=True)
    _,  sorter = torch.sort(eival[:,0], descending=True)
    return mean, eivect[:,sorter].t()


In [19]:
mean, eibase = PCA(train_input)
eibase.shape

torch.Size([784, 784])

In [34]:
torch.manual_seed(1)
for c in [ False, True ]:

    train_input, train_target, test_input, test_target = prologue.load_data(cifar = c)

    # Original test on dataset. Define baseline

    nb_errors, perc_errors = compute_nb_errors(train_input, train_target, test_input, test_target)
    print(f'Original errors: nb_errors {nb_errors} error {100*perc_errors:.02f}%')

    # Project onto 100d random basis, that is a set of d 100-elem long vectors. 
    # the basis defines the projection matrix.

    basis = train_input.new(100, train_input.size(1)).normal_()

    nb_errors, perc_errors = compute_nb_errors(train_input, train_target, test_input, test_target, None, basis)
    print(f'Random basis: nb_errors {nb_errors} error {100*perc_errors:.02f}%')

    # Project onto PCA space with the first npc PCs taken into account

    mean, basis = PCA(train_input)
    
    for npc in [ 100, 50, 10, 3 ]:
        nb_errors, perc_errors = compute_nb_errors(train_input, train_target, test_input, test_target, mean, basis[:npc])
        print(f'PCA {npc:d}d nb_errors {nb_errors} error {100*perc_errors:.02f}%')

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
Original errors: nb_errors 172 error 17.20%
Random basis: nb_errors 210 error 21.00%
PCA 100d nb_errors 164 error 16.40%
PCA 50d nb_errors 155 error 15.50%
PCA 10d nb_errors 214 error 21.40%
PCA 3d nb_errors 597 error 59.70%
* Using CIFAR
Files already downloaded and verified
Files already downloaded and verified
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
Original errors: nb_errors 746 error 74.60%
Random basis: nb_errors 776 error 77.60%
PCA 100d nb_errors 745 error 74.50%
PCA 50d nb_errors 737 error 73.70%
PCA 10d nb_errors 763 error 76.30%
PCA 3d nb_errors 839 error 83.90%
